In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import torch
import transformers
import inspect
import time
import logging
import tmunlp as nlp

from tqdm import trange, tqdm, tqdm_notebook, tqdm_pandas, tqdm_gui
from datetime import datetime
from tqdm import tqdm
from transformers import BertConfig, BertModel, BertTokenizer, BertForSequenceClassification, AdamW, BertPreTrainedModel
from transformers import get_constant_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report, accuracy_score, f1_score
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#torch.cuda.get_device_name(0)


C:\Users\MrBonBon\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\MrBonBon\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\MrBonBon\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\MrBonBon\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Futu

'GeForce GTX 1070'

# 1. Load data & pre-processing

In [2]:
def preprocessing(df):
    """
    Preprocessing step
    As above dataframe heads, there is a lot of <br /> character 
    """
    df.sentence = df.sentence.str.replace('<br />','')
    df.sentence = df.sentence.str.lower()
    return df

train = pd.read_csv('./dataset/imdb/train.csv')
test = pd.read_csv('./dataset/imdb/test.csv')

train = preprocessing(train)
test = preprocessing(test)

print('Train data:\n{}\n\nTest data:\n{}'.format(train.head(5), test.head(5)))

Train data:
                                            sentence  sentiment  polarity
0  this is a very bland and inert production of o...          2         0
1  i've seen this film in avant-premiere at imagi...          7         1
2  revolt of the zombies (2 outta 5 stars) no, th...          4         0
3  may contain minor spoilers.dressed to kill, ha...          7         1
4  (spoilers)i shoulda figured. the dvd didn't ev...          2         0

Test data:
                                            sentence  sentiment  polarity
0  i loved this movie so much. i'm a big fan of a...         10         1
1  the stark, cold landscape of big sky country, ...          9         1
2  this cheapo exploitation flick is some genuine...          2         0
3  this movie has been promoting in everywhere in...          1         0
4  this is a great off-the-wall romantic comedy a...          8         1


# 2. Build model

In [3]:
class BertModelBonz():
    def __init__(self, load_model=None, load_config=None, model='bert-base-uncased', max_len=512, batch_size=6):
        self.pre_trained_model = model
        self.config = load_config
        self.max_len = self.config.max_position_embeddings
        self.batch_size = batch_size
        
        self.tokenizer = BertTokenizer.from_pretrained(self.pre_trained_model)
        self.tokenizer.max_len = max_len
        
        self.model = BertForSequenceClassification(config=self.config)
        #self.model = BertForSequenceClassification.from_pretrained(self.pre_trained_model, config=self.config)
        self.optimizer = AdamW(params = self.model.parameters(), lr=1e-5)
    
    def create_ids(self, sentences):
        logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR) #Disable tokenizer logs, it's really annoy
        input_ids = []
        for sen in tqdm_notebook(sentences, desc="Create Ids"):
            tmp = self.tokenizer.encode(sen)
            input_ids.append(tmp)
        input_ids = pad_sequences(input_ids, 
                                  maxlen=self.max_len, 
                                  dtype='int64', 
                                  truncating='post', 
                                  padding='post')
        return input_ids
    
    def prepare_data(self, input_ids, input_labels=None):
        input_ids = torch.tensor(self.create_ids(input_ids))
        if input_labels is None:
            return DataLoader(TensorDataset(input_ids), 
                              batch_size=self.batch_size)
        else:
            input_labels = torch.tensor(input_labels)
            return DataLoader(TensorDataset(input_ids, input_labels), 
                              batch_size=self.batch_size)
        
    def flat_accuracy(self, preds, labels):
        pred_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    def train(self, dataloader, epochs=4):
        self.train_loss_set =[]
        for i in trange(epochs, desc="Epoch"):
            # Training model
            self.model.to(device)
            self.model.train()
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            for input_ids, input_labels in tqdm_notebook(dataloader):
                self.optimizer.zero_grad()
                loss = self.model(input_ids=input_ids.cuda(), labels=input_labels.cuda())[0]
                self.train_loss_set.append(loss)    
                loss.backward()
                self.optimizer.step()
                
                tr_loss += loss.item()
                nb_tr_examples += input_ids.size(0)
                nb_tr_steps += 1
            print("Train loss: {}".format(tr_loss/nb_tr_steps))

            # Evaluation
            self.model.eval()
            eval_loss, eval_accuracy = 0, 0
            nb_eval_steps, nb_eval_examples = 0, 0
            for input_ids, input_labels in dataloader:
                with torch.no_grad():
                    logits = self.model(input_ids.cuda())[0]
                logits = logits.detach().cpu().numpy()
                label_ids = input_labels.to('cpu').numpy()
                tmp_eval_accuracy = self.flat_accuracy(logits, label_ids)
                eval_accuracy += tmp_eval_accuracy
                nb_eval_steps += 1
            print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
            
            '''
            #Save model for each epoch:
            filename = 'bert_512_epoch'+str(i)+'.sd'
            filepath = './model/bert/'+filename
            torch.save(self.model.state_dict(), filepath)
            '''
    def generate_cls_vectors(self, dataloader):
        self.model.to('cuda')
        self.model.eval()
        cls_vectors = []
        for input_ids, input_labels in tqdm_notebook(dataloader):
            with torch.no_grad():
                outputs = self.model(input_ids.cuda())
                last_hidden_layer = outputs[1][12]
                cls_vector = last_hidden_layer[:,0,:]
            cls_vector = cls_vector.detach().cpu().numpy()
            cls_vectors.extend(cls_vector)
        return cls_vectors
                
    
    def predict(self, test_data, test_labels):
        test_ids = self.create_ids(test_data)
        test_inputs = torch.tensor(test_ids)
        test_dataloader = DataLoader(test_inputs, batch_size=self.batch_size)
        
        # Preditcion
        self.model.to(device)
        self.model.eval()
        self.predictions = []
        for input_ids in tqdm_notebook(test_dataloader, desc="Predicting"):
            with torch.no_grad():
                logits = self.model(input_ids.to(device))[0] #This is for generate predict only
            logits = logits.detach().cpu().numpy()
            self.predictions.append(logits)
        return self.predictions
        self.predictions = [j for i in self.predictions for j in i]
        self.predictions = np.argmax(self.predictions, axis=1)
        print(classification_report(self.predictions, test_labels, digits=4))

        
        
#Create model
config = BertConfig(output_hidden_states=True, max_position_embeddings=512)
bert_model = BertModelBonz(load_config=config, batch_size=6)
bert_model.model.config

{
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": true,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

## 2.1. Create train & test dataloader

In [4]:
NUM_DATA = 25000

train_dataloader = bert_model.prepare_data(input_ids=train.sentence[:NUM_DATA], input_labels=train.polarity[:NUM_DATA])
test_dataloader = bert_model.prepare_data(input_ids=test.sentence[:NUM_DATA], input_labels=test.polarity[:NUM_DATA])

## 2.2. Train model

In [6]:
#Train model

bert_model.train(train_dataloader)
#torch.save(bert_model.model, 'bert_eb1024_1e5_e4.pth')




Epoch:   0%|                                                                                     | 0/4 [00:00<?, ?it/s]


Train loss: 0.704374572213445
Validation Accuracy: 0.49998000159987244


Epoch:  25%|██████████████████▎                                                      | 1/4 [30:56<1:32:49, 1856.49s/it]


Train loss: 0.7009197178506611
Validation Accuracy: 0.49998000159987244


Epoch:  50%|███████████████████████████████████▌                                   | 2/4 [1:01:51<1:01:52, 1856.15s/it]


Train loss: 0.7004017656530898
Validation Accuracy: 0.49998000159987244


Epoch:  75%|██████████████████████████████████████████████████████▊                  | 3/4 [1:32:40<30:53, 1853.82s/it]


Train loss: 0.6998716865851492
Validation Accuracy: 0.49998000159987244


Epoch: 100%|█████████████████████████████████████████████████████████████████████████| 4/4 [2:03:29<00:00, 1852.49s/it]


## 2.3. Predict test data

In [8]:
for i in range(4):
    filename = 'bert_512_epoch'+str(i)+'.sd'
    filepath = './model/bert/'+filename
    bert_model.model.load_state_dict(torch.load(filepath))
    bert_model.model.eval()
    print('Model ',filename,':')
    bert_model.predict(test_data=test.sentence, test_labels=test.polarity)

Model  bert_512_epoch0.sd :



              precision    recall  f1-score   support

           0     0.9084    0.9448    0.9262     12019
           1     0.9469    0.9118    0.9290     12981

   micro avg     0.9276    0.9276    0.9276     25000
   macro avg     0.9276    0.9283    0.9276     25000
weighted avg     0.9284    0.9276    0.9277     25000

Model  bert_512_epoch1.sd :



              precision    recall  f1-score   support

           0     0.9376    0.9228    0.9301     12701
           1     0.9215    0.9366    0.9290     12299

   micro avg     0.9296    0.9296    0.9296     25000
   macro avg     0.9296    0.9297    0.9296     25000
weighted avg     0.9297    0.9296    0.9296     25000

Model  bert_512_epoch2.sd :



              precision    recall  f1-score   support

           0     0.9002    0.9581    0.9282     11744
           1     0.9606    0.9059    0.9324     13256

   micro avg     0.9304    0.9304    0.9304     25000
   macro avg     0.9304    0.9320    0.9303     25000
weighted avg     0.9322    0.9304    0.9305     25000

Model  bert_512_epoch3.sd :



              precision    recall  f1-score   support

           0     0.9211    0.9444    0.9326     12192
           1     0.9458    0.9230    0.9343     12808

   micro avg     0.9334    0.9334    0.9334     25000
   macro avg     0.9334    0.9337    0.9334     25000
weighted avg     0.9337    0.9334    0.9335     25000



## 2.4. Generate CLS vectors

In [50]:
# Load state dict from trained model
filepath = './model/bert/bert_512_epoch2.sd'
bert_model.model.load_state_dict(torch.load(filepath))

cls_vectors = bert_model.generate_cls_vectors(train_dataloader)
cls_vectors_test = bert_model.generate_cls_vectors(test_dataloader)

# 3. Feature Extraction with TMUNLP

## 3.1. Build negative & positive vectors

In [24]:
result = nlp.get_label_term_weighting('tmunlp_file.txt', ['0','1'])
VECTOR_LEN = 50

#Create negative word list
negative_list =  nlp.get_keyword('0', result, VECTOR_LEN)
temp_max = negative_list[list(negative_list.keys())[0]]
for i in negative_list.keys():
    negative_list[i] = negative_list[i] #/ temp_max


#Create postive word list
positive_list =  nlp.get_keyword('1', result, VECTOR_LEN)
temp_max = positive_list[list(positive_list.keys())[0]]
for i in positive_list.keys():
    positive_list[i] = positive_list[i] #/ temp_max



def embedding_1hot(df, words):
    arr = []
    for i in range(df.shape[0]):
        temp = [words[word] if word in df.sentence[i] else 0 for word in words.keys()]
        arr.append(temp)
    return arr

train['negative_embedding'] = embedding_1hot(train, negative_list)
train['positive_embedding'] = embedding_1hot(train, positive_list)

test['negative_embedding'] = embedding_1hot(test, negative_list)
test['positive_embedding'] = embedding_1hot(test, positive_list)


## 3.2. Build bert_llr model

In [29]:
class BertLLR(nn.Module):
    def __init__(self, VECTOR_LEN):
        super(BertLLR, self).__init__()
        self.bert = nn.Linear(768,768)
        self.bert_activation = nn.Tanh()
        
        self.llr = nn.Linear(2*VECTOR_LEN, 2*VECTOR_LEN)
        self.llr_activation = nn.Tanh()
        
        self.dropout = nn.Dropout(0.1)
        
        self.classifier = nn.Linear(768+2*VECTOR_LEN, 2)
    
    def forward(self, cls_vectors=None, neg_embed=None, pos_embed=None, labels=None):
        tanh_cls_vectors = self.bert_activation(self.bert(cls_vectors))
        
        llr_vectors = torch.cat([neg_embed, pos_embed], dim=1)
        tanh_llr_vectors = self.llr_activation(self.llr(llr_vectors))
        
        concat_vectors = torch.cat([tanh_cls_vectors, tanh_llr_vectors], dim=1)
        concat_vectors = self.dropout(concat_vectors)
        
        logits = self.classifier(concat_vectors)
        
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))
            outputs = (logits, loss)
        else:
            outputs = (logits,)
        
        return outputs
    
bert_llr_model = BertLLR(VECTOR_LEN)
bert_llr_optimizer = torch.optim.Adam(params = bert_llr_model.parameters(), 
                                      lr = 1e-4
                                     )


bert_llr_model   

BertLLR(
  (bert): Linear(in_features=768, out_features=768, bias=True)
  (bert_activation): Tanh()
  (llr): Linear(in_features=100, out_features=100, bias=True)
  (llr_activation): Tanh()
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Linear(in_features=868, out_features=2, bias=True)
)

## 3.3. Train bert_llr model

In [31]:
dataloader = DataLoader(TensorDataset(torch.tensor(cls_vectors, dtype=torch.float),
                                      torch.tensor(train.negative_embedding, dtype=torch.float),
                                      torch.tensor(train.positive_embedding, dtype=torch.float),
                                      torch.tensor(train.polarity)
                                     ),
                        batch_size=16
                       )

dataloader_test = DataLoader(TensorDataset(torch.tensor(cls_vectors_test, dtype=torch.float),
                                          torch.tensor(test.negative_embedding, dtype=torch.float),
                                          torch.tensor(test.positive_embedding, dtype=torch.float),
                                          torch.tensor(test.polarity)
                                         ),
                            batch_size=16
                           )


bert_llr_model.to('cuda')
train_loss = []
train_accuracy = []
test_f1 = []
for _ in range(10):
    # Training
    bert_llr_model.train()
    tr_loss = []
    for cls_vector, neg_embed, pos_embed, labels in tqdm(dataloader, desc='Training'):
        #print(labels)
        bert_llr_optimizer.zero_grad()
        logits, loss = bert_llr_model(cls_vectors = cls_vector.cuda(), 
                                      neg_embed = neg_embed.cuda(), 
                                      pos_embed = pos_embed.cuda(), 
                                      labels = labels.cuda())
        loss.backward()
        bert_llr_optimizer.step()
        tr_loss.append(loss.item())
    loss_score = sum(tr_loss)/len(tr_loss)
    train_loss.append(loss_score)
    
    # Evaluation
    bert_llr_model.eval()
    predictions = []
    for cls_vector, neg_embed, pos_embed, labels in tqdm(dataloader, desc='Predicting'):
        with torch.no_grad():
            logits = bert_llr_model(cls_vectors = cls_vector.cuda(),
                                    neg_embed = neg_embed.cuda(),
                                    pos_embed = pos_embed.cuda())[0] #This is for generate predict only
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    predictions = [j for i in predictions for j in i]
    predictions = np.argmax(predictions, axis=1)
    acc_score = accuracy_score(predictions, train.polarity)
    train_accuracy.append(acc_score)
    
    # Predict data
    predictions = []
    for cls_vector, neg_embed, pos_embed, labels in tqdm(dataloader_test, desc='Predicting'):
        with torch.no_grad():
            logits = bert_llr_model(cls_vectors = cls_vector.cuda(),
                                    neg_embed = neg_embed.cuda(),
                                    pos_embed = pos_embed.cuda())[0] #This is for generate predict only
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    preds = [j for i in predictions for j in i]
    preds = np.argmax(preds, axis=1)
    test_f1.append(f1_score(preds, test.polarity, average='macro'))
    
    # Print result
    print('Train loss: ', loss_score)
    print('Train accuracy: ',acc_score)
    print(classification_report(preds, test.polarity, digits=4))
        

Predicting: 100%|█████████████████████████████████████████████████████████████████| 1563/1563 [00:01<00:00, 883.03it/s]


Train loss:  0.3095645647965519
Train accuracy:  0.86768
              precision    recall  f1-score   support

           0     0.8982    0.8089    0.8513     13880
           1     0.7878    0.8856    0.8339     11120

   micro avg     0.8430    0.8430    0.8430     25000
   macro avg     0.8430    0.8473    0.8426     25000
weighted avg     0.8491    0.8430    0.8435     25000



Predicting: 100%|█████████████████████████████████████████████████████████████████| 1563/1563 [00:01<00:00, 965.24it/s]


Train loss:  0.30776681377053716
Train accuracy:  0.8684
              precision    recall  f1-score   support

           0     0.9023    0.8063    0.8516     13988
           1     0.7833    0.8891    0.8329     11012

   micro avg     0.8428    0.8428    0.8428     25000
   macro avg     0.8428    0.8477    0.8422     25000
weighted avg     0.8499    0.8428    0.8434     25000



Predicting: 100%|█████████████████████████████████████████████████████████████████| 1563/1563 [00:01<00:00, 965.61it/s]


...

Train loss:  0.2075112884839185
Train accuracy:  0.91696
              precision    recall  f1-score   support

           0     0.9006    0.8126    0.8543     13853
           1     0.7923    0.8885    0.8377     11147

   micro avg     0.8464    0.8464    0.8464     25000
   macro avg     0.8464    0.8505    0.8460     25000
weighted avg     0.8523    0.8464    0.8469     25000



Predicting: 100%|█████████████████████████████████████████████████████████████████| 1563/1563 [00:01<00:00, 945.28it/s]


Train loss:  0.2060674474510869
Train accuracy:  0.91712
              precision    recall  f1-score   support

           0     0.8966    0.8164    0.8546     13729
           1     0.7983    0.8854    0.8396     11271

   micro avg     0.8475    0.8475    0.8475     25000
   macro avg     0.8475    0.8509    0.8471     25000
weighted avg     0.8523    0.8475    0.8478     25000



Training:  77%|███████████████████████████████████████████████████▊               | 1210/1563 [00:07<00:02, 127.44it/s]


KeyboardInterrupt: 

In [27]:
print(train_loss)
print(train_accuracy)
print(test_f1)

[0.6091443893624207, 0.5192290250490814, 0.47306699867783947, 0.4451642018228636, 0.4262837194435427, 0.41259511840015517, 0.40247247462957547, 0.3933825103026205, 0.38607654931239416, 0.3801700164369109]
[0.75452, 0.78908, 0.80148, 0.81176, 0.82024, 0.82596, 0.8304, 0.83424, 0.83784, 0.84064]
[0.7431020667888952, 0.7807754945986536, 0.7938288052732954, 0.8032311831550436, 0.8101958010506023, 0.8153445873109721, 0.82003608808, 0.8238238625140057, 0.8256383482518476, 0.8286212187232584]


## 3.4. Predict test data

In [70]:
dataloader = DataLoader(TensorDataset(torch.tensor(cls_vectors_test, dtype=torch.float),
                                      torch.tensor(test.negative_embedding, dtype=torch.float),
                                      torch.tensor(test.positive_embedding, dtype=torch.float),
                                      torch.tensor(test.polarity)
                                     ),
                        batch_size=16
                       )

bert_llr_model.to('cuda')
bert_llr_model.eval()

predictions = []
for cls_vector, neg_embed, pos_embed, labels in tqdm_notebook(dataloader, desc='Predicting'):
    with torch.no_grad():
        logits = bert_llr_model(cls_vectors = cls_vector.cuda(),
                                neg_embed = neg_embed.cuda(),
                                pos_embed = pos_embed.cuda())[0] #This is for generate predict only
    logits = logits.detach().cpu().numpy()
    predictions.append(logits)


preds = [j for i in predictions for j in i]
preds = np.argmax(preds, axis=1)
print(classification_report(preds, test.polarity, digits=4))




              precision    recall  f1-score   support

           0     0.9366    0.9342    0.9354     12533
           1     0.9340    0.9365    0.9352     12467

   micro avg     0.9353    0.9353    0.9353     25000
   macro avg     0.9353    0.9353    0.9353     25000
weighted avg     0.9353    0.9353    0.9353     25000

